In [8]:
import pandas as pd
import numpy as np

data = pd.read_csv('IFR_Extract_with_selected_columns_15-5-23.csv')

obreak_date = pd.to_datetime(data.obreak_date)
datebone = pd.to_datetime(data.datebone)
y = ( abs( datebone - obreak_date))
X = pd.DataFrame({
    "PatientAge": data.PatientAge,
    "PatientGender": data.PatientGender,
    
})
y

0       524 days
1      2046 days
2     15455 days
3      4354 days
4      2207 days
         ...    
795     579 days
796    5109 days
797    2125 days
798     518 days
799     483 days
Length: 800, dtype: timedelta64[ns]

In [9]:

# X = data.drop(['PostalCode','DateSurveyed','obreak_date','datebone','obone_text','rxother_text'
# ,'rx2_other_text','norxo_text','obone_text','obreak_obone_text'],axis=1)

dropList = []
for i in data:
    if data[i].dtypes == 'O':
        dropList.append(data[i].name)
dropList.append("CompletedSurveyId")
dropList.append("PatientId")
X = data.drop(dropList,axis=1)
X.fillna(0,inplace=True)
y = pd.DataFrame({"time":y})


In [11]:
y.time

y['event'] = y.time.apply(lambda x: x.days != 0 )
structured_array = y.to_records(index=False)

swapped = pd.DataFrame({
    "event": y.event,
    "time": y.time.apply(lambda x: x.days)
})
(swapped.time < 100).value_counts()

False    751
True      49
Name: time, dtype: int64

In [12]:
swapped.event = swapped.event.astype(bool)
swapped.event
structured_array = np.rec.array(swapped.to_records(index=False))

In [13]:
swapped

,event,time
0,True,524
1,True,2046
2,True,15455
3,True,4354
4,True,2207
...,...,...
795,True,579
796,True,5109
797,True,2125
798,True,518


In [8]:
import pandas as pd
import numpy as np
from lifelines import CoxPHFitter
from lifelines.utils import k_fold_cross_validation
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler

# X: input features, y: survival outcomes

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Perform LASSO feature selection
lasso = LassoCV(cv=5)
lasso.fit(X_scaled, swapped.time)
selected_features = lasso.coef_ != 0

# Filter the selected feature names
feature_names = X.columns[selected_features].tolist()

# Create a new DataFrame with selected features and survival outcomes
selected_data = pd.concat([swapped[['time', 'event']], X.loc[:, feature_names]], axis=1)

# Fit CoxPH model on the selected features
cph = CoxPHFitter()
cph.fit(selected_data, duration_col='time', event_col='event')

# Assess model performance using cross-validation
c_index_values = k_fold_cross_validation(cph, selected_data, duration_col='time', event_col='event', k=5, scoring_method='concordance_index')
c_index_mean = np.mean(c_index_values)
print("Cross-validated C-index: {:.3f}".format(c_index_mean))




Cross-validated C-index: 0.630


In [19]:
from pycaret.datasets import get_data
from pycaret.regression import *
mergedData = pd.concat([X,swapped.time],axis=1)
clf = setup(mergedData, target='time',feature_selection=True)


,Description,Value
0,Session id,2543
1,Target,time
2,Target type,Regression
3,Original data shape,"(800, 190)"
4,Transformed data shape,"(800, 38)"
5,Transformed train set shape,"(560, 38)"
6,Transformed test set shape,"(240, 38)"
7,Numeric features,189
8,Preprocess,True
9,Imputation type,simple


In [20]:

best_model = compare_models()


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,14:47:29
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library
